# Исследование надежности заёмщика

Заказчиком выступает кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Цель исследования 

Проверка четырех гипотез:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"-4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"-5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"-4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,-926.19,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"-2,879.20",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,-152.78,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"-6,929.87",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"-2,188.76",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == i), 'total_income'].median()

### Обработка аномальных значений
В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs()

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях.

In [7]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный       366,413.65
в декрете           3,296.76
госслужащий         2,689.37
компаньон           1,547.38
пенсионер         365,213.31
предприниматель       520.85
сотрудник           1,574.20
студент               578.75
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
#Выведем перечень уникальных значений столбца children.
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце children есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма data.

In [9]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
#проверем результат
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков

Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.

In [11]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == i), 'days_employed'].median()

In [12]:
#проверим
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце total_income на целочисленный

In [13]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

В столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Length: 21402, dtype: bool

In [16]:
data = data.drop_duplicates().reset_index(drop=True)

### Категоризация данных
0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

In [17]:
# создайте функцию categorize_income()
def categorize_income(row):
    if row <= 30000:
        return 'E'
    if row <= 50000:
        return 'D'
    if row <= 200000:
        return 'C'
    if row <= 1000000:
        return 'B'
    if row >= 1000001:
        return 'A'

In [18]:
# примените функцию методом apply()
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Cоздайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

In [20]:
# создайте функцию categorize_purpose()
def categorize_purpose(row):
    if 'автом' in row:
        return 'операции с автомобилем'
    if ('жил' in row) or ('нед' in row):
        return 'операции с недвижимостью'
    if 'свад' in row:
        return 'проведение свадьбы'
    if 'образов' in row:
        return 'получение образования'

In [21]:
# примените функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_children = data.pivot_table(index='children', values='debt', aggfunc=['count','sum', 'mean'])
data_children = data_children.rename(columns={'count': 'все клиенты', 'sum': 'количество должников', 'mean': 'доля должников',})
data_children['доля должников'] = data_children['доля должников'] * 100
data_children

,все клиенты,количество должников,доля должников
,debt,debt,debt
children,,,
0,14091,1063,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


**Вывод:** Если предположить что данные для 3 детей недостаточно точны. Можно сказать что увеличение количества детей приводит к росту доли должников. Однако рост небольшой в районе 0.2 процента. А показатели клиентов без детей ниже на 1 - 1,5 %

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_family = data.pivot_table(index='family_status', values='debt', aggfunc=['count','sum', 'mean'])
data_family = data_family.rename(columns={'count': 'все клиенты', 'sum': 'количество должников', 'mean': 'доля должников',})
data_family['доля должников'] = data_family['доля должников'] * 100
display(data_family)
data_family = data_family['доля должников', 'debt'].sort_values(ascending=False)
data_family

,все клиенты,количество должников,доля должников
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.76
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56


family_status
Не женат / не замужем   9.76
гражданский брак        9.31
женат / замужем         7.56
в разводе               7.06
вдовец / вдова          6.62
Name: (доля должников, debt), dtype: float64

**Вывод:** Зависимость есть. Отдельные категории такие как 'вдовец / вдова', 'в разводе' имеют лучшие показатели в то время как 'Не женат / не замужем' худшие.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#Группы А, D, E имеют слишко мало данных. Их мы рассматривать не будем. Рассмотрим подробней группы С и В.
#Оставшиеся данные перераспределим на меньшие группы чтобы более точно определить разницу в самой большой группе.
def categorize_income2(income):
    try:
        if 50000 <= income <= 100000:
            return '1. от 50к ло 100к'
        elif 100001 <= income <= 150000:
            return '2. от 100к до 150к'
        elif 150001 <= income <= 200000:
            return '3. от 150к до 200к'
        elif 200001 <= income <= 400000:
            return '4. от 200к до 400к'
        elif 400001 <= income <= 1000000:
            return '5. от 400к'
    except:
        pass

In [25]:
data['total_income_category2'] = data['total_income'].apply(categorize_income2)

In [26]:
data_income2 = data.pivot_table(index='total_income_category2', values='debt', aggfunc=['count','sum', 'mean'])
data_income2 = data_income2.rename(columns={'count': 'все клиенты', 'sum': 'количество должников', 'mean': 'доля должников',})
data_income2['доля должников'] = data_income2['доля должников'] * 100
data_income2

,все клиенты,количество должников,доля должников
,debt,debt,debt
total_income_category2,,,
1. от 50к ло 100к,4073,331,8.13
2. от 100к до 150к,7110,619,8.71
3. от 150к до 200к,4738,403,8.51
4. от 200к до 400к,4511,325,7.20
5. от 400к,503,29,5.77


**Вывод:** Клиенты с доходом от 200000 имеют меньше долгов по кредитам. 

#### Как разные цели кредита влияют на его возврат в срок?

In [27]:
data_purpose = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count','sum', 'mean'])
data_purpose = data_purpose.rename(columns={'count': 'все клиенты', 'sum': 'количество должников', 'mean': 'доля должников',})
data_purpose['доля должников'] = data_purpose['доля должников'] * 100
data_purpose

,все клиенты,количество должников,доля должников
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35
операции с недвижимостью,10751,780,7.26
получение образования,3988,369,9.25
проведение свадьбы,2313,183,7.91


**Вывод:** Категории операции с автомобилем и получение образования выделяються повышенной долей должников

#### Приведите возможные причины появления пропусков в исходных данных.

Ответ: Пропущенные данные касались общего трудового стажа в днях и ежемесячного дохода. Что может означать что люди работают неофициально или безработные.

####  Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Ответ: Медианное значение лучше характеризует данные, когда некоторые значения сильно выделяются среди большинства.

## Выводы 
В ходе иследования был выявлен ряд зависимостей. 
1. С увеличением количества детей, увеличивается вероятность просрочки.
2. Клиенты, побывавшие в браке, реже допускают просрочки по кредитам.
3. Клиенты с доходом от 200000 имеют меньше долгов по кредитам. 
4. Кредиты на недвижимость чаще выплачивают вовремя.